심심해서 만드는 AI챗봇(TextBlob)
영문으로 인식함

In [ ]:
from textblob import TextBlob
import nltk

def analyze_emotion(text: str) -> float:
  """
  text를 받아 polarity를 float으로 반환 합니다.
  polarity 범위: -1.0~ 1.0
  """
  blob = TextBlob(text)
  sentiment = float(blob.sentiment.polarity) # -1(부정) ~ +1(긍정)
  return sentiment

def generate_response(sentiment):
  if sentiment > 0.3:
    return "기분이 좋아보이네요! 좋은 하루 보내세용"
  elif sentiment < -0.3:
    return "오늘 힘든일이 있었나요? 조금 쉬어가도 돼요"
  else:
    return "그냥 그런 날도 있죠. 포근한 이불속에 들어가 보아요."
  
while True:
  user_input = input("당신 : ")
  if user_input.lower() in ["exit", "quit", "종료"]:
    print("봇: 대화해줘서 고마워요. 좋은 하루 보내세요!😊")
    break
  sentiment = analyze_emotion(user_input)
  response = generate_response(sentiment)
  print(f"봇: {response}")

  # pip install textblob

기존 만든 챗봇에 NLTK 추가하기 (텍스트 전처리 + 감정 단어 기반 분석)

In [ ]:
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

# nltk 다운로드 (최초 1회만)
# nltk.download('punkt')
# nltk.download('stopwords')

# 간단한 감정 단어 사전 (원하면 확장 가능)
positive_words = ['좋아', '행복', '즐겁', '감사', '기분좋', '웃음', '사랑']
negative_words = ['싫', '짜증', '우울', '힘들', '피곤', '화나', '괴롭']

def analyze_emotion_with_nltk(text):
    # 기본 polarity 계산 (TextBlob)
    blob = TextBlob(text)
    base_score = blob.sentiment.polarity

    # NLTK 토큰화 + 불용어 제거
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))  # 영어용, 한국어는 별도 사전 필요
    filtered = [t for t in tokens if t.lower() not in stop_words]

    # 감정 단어 기반 점수 조정
    extra_score = 0
    for word in filtered:
        for pos in positive_words:
            if pos in word:
                extra_score += 0.1
        for neg in negative_words:
            if neg in word:
                extra_score -= 0.1

    final_score = max(-1.0, min(1.0, base_score + extra_score))
    return final_score


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import platform
print(platform.architecture())

In [2]:
from konlpy.tag import Okt

# 에러 없이 객체가 생성되어야 합니다.
okt = Okt() 
print("Okt 객체 생성 성공! Konlpy 사용 준비 완료.")

Okt 객체 생성 성공! Konlpy 사용 준비 완료.


In [3]:
from konlpy.tag import Okt
import nltk
import jpype
from textblob import TextBlob

# 필요시 다운로드 (영문 불용어 제거용)
# nltk.download('stopwords')
# jvm_path = r"C:\dev\eclipse\plugins\org.eclipse.justj.openjdk.hotspot.jre.full.win32.x86_64_21.0.3.v20240426-1530\jre\bin\server\jvm.dll"


okt = Okt()

# 간단한 한국어 감정 단어 사전 (원하면 계속 추가 가능)
positive_words = ['좋아', '행복', '즐겁', '감사', '기분좋', '웃음', '사랑', '기쁨', '축하']
negative_words = ['싫', '짜증', '우울', '힘들', '피곤', '화나', '괴롭', '눈물', '지침']

def analyze_emotion_korean(text):
    """
    한국어 형태소 분석 기반 감정 점수 계산
    1) 기본 TextBlob polarity (영문 기반, 0으로 둬도 무방)
    2) 한국어 감정 단어를 기반으로 추가 점수 계산
    """
    # 1️⃣ TextBlob 기본 점수 (한국어는 거의 0)
    try:
        blob = TextBlob(text)
        base_score = blob.sentiment.polarity
    except Exception:
        base_score = 0.0

    # 2️⃣ 형태소 분석
    tokens = okt.morphs(text)
    extra_score = 0

    # 3️⃣ 감정 단어별 점수 계산
    for word in tokens:
        for pos in positive_words:
            if pos in word:
                extra_score += 0.2
        for neg in negative_words:
            if neg in word:
                extra_score -= 0.2

    # 점수 합산 및 범위 제한
    final_score = max(-1.0, min(1.0, base_score + extra_score))
    return final_score


def generate_response(sentiment: float) -> str:
    if sentiment >= 0.6:
        return "와, 오늘 기분이 아주 좋으시네요! 😊 행복한 하루 보내세요!"
    elif sentiment > 0.3:
        return "기분이 좋아보이네요! 좋은 하루 보내시길 바랄게요 🌸"
    elif sentiment >= -0.3:
        return "그냥 그런 날도 있죠. 따뜻한 차 한잔 어때요? ☕"
    elif sentiment >= -0.6:
        return "조금 지친 하루인가요? 괜찮아요, 잠깐 쉬어가도 됩니다 🌙"
    else:
        return "많이 힘들어 보이네요. 괜찮아요. 오늘은 스스로를 조금 더 보듬어주세요 💛"


def main():
    print("==== 💬 한국어 감정 챗봇 ====")
    print("텍스트를 입력하면 감정을 분석합니다.")
    print("종료하려면 'exit' 또는 '종료'를 입력하세요.\n")

    while True:
        user_input = input("당신: ").strip()
        if not user_input:
            print("봇: 아무 말도 안 하셨네요. 😅")
            continue

        if user_input.lower() in ("exit", "종료", "quit"):
            print("봇: 대화해줘서 고마워요. 좋은 하루 보내세요! 🌷")
            break

        # 감정 분석 실행
        sentiment = analyze_emotion_korean(user_input)
        print(f"(디버그) 감정 점수: {sentiment:.2f}")

        # 반응 생성
        response = generate_response(sentiment)
        print(f"봇: {response}\n")


if __name__ == "__main__":
    main()


==== 💬 한국어 감정 챗봇 ====
텍스트를 입력하면 감정을 분석합니다.
종료하려면 'exit' 또는 '종료'를 입력하세요.

(디버그) 감정 점수: 0.20
봇: 그냥 그런 날도 있죠. 따뜻한 차 한잔 어때요? ☕

(디버그) 감정 점수: -0.20
봇: 그냥 그런 날도 있죠. 따뜻한 차 한잔 어때요? ☕

봇: 대화해줘서 고마워요. 좋은 하루 보내세요! 🌷
